# To Contracept or Not to Contracept

Import packeges and py-files

In [10]:
# a. packeges 
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import copy

# b. py-files 
from Model import child_model 
import Solve as solver
import Estimate as estimate

## Read data

In [11]:
model = child_model()
dta = model.read_data()

## State trantision contional on age and choice

In [12]:
solver.P_list(model, dta)

AttributeError: module 'Solve' has no attribute 'P_list'

## Estimate

In [ ]:
samplesize = dta.shape[0]
par_mu = copy.copy(model.mu)
par_eta1 = copy.copy(model.eta1)
par_eta2 = copy.copy(model.eta2)

I = 50

log_lik = np.nan + np.zeros((I,I,I))
mu = np.linspace(0,1.8,I)
eta1 = np.linspace(0,1.8,I)
eta2 = np.linspace(-0.9,0.9,I)
iter = 0

for i in range(I):
    for j in range(I):
        for k in range(I):
                
                # Estimate parameters
                pnames = ['eta1', 'eta2', 'mu']
                theta = np.array([eta1[i], eta2[j], mu[k]])
                log_lik[i,j,k]=estimate.ll(theta, model, solver, dta, pnames)
                print(iter)
                iter = iter+1

log_lik *= samplesize*(-1)

# Re-inset the true parameters 
model.eta1 = copy.copy(par_eta1)
model.eta2 = copy.copy(par_eta2)
model.mu = copy.copy(par_mu)

x, y, z = np.unravel_index(np.argmax(log_lik), log_lik.shape)
print(eta1[x], eta2[y], mu[z])